In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
from dash import Dash, html
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import yaml
import s3fs

# enabling conexion to s3 storage for data retrieving
credentials = yaml.safe_load(open('/home/coder/.config/code-server/telco_churn/config.yaml', 'rb').read())

fs = s3fs.S3FileSystem(
  client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
  key=credentials['input']['key_id'], secret=credentials['input']['access_key'],
  token=credentials['input']['token'])

comms_df = pd.read_sas(fs.open("ikonkobo/commsdata/commsdata.sas7bdat"), format='sas7bdat')

In [17]:
comms_df.head(10)

,Customer_ID,upsell_xsell,churn,lifetime_value,avg_arpu_3m,acct_age,billing_cycle,nbr_contracts_ltd,credit_class,sales_channel,...,pymts_late_ltd,calls_care_ltd,MB_Data_Usg_M04,MB_Data_Usg_M05,MB_Data_Usg_M06,MB_Data_Usg_M07,MB_Data_Usg_M08,MB_Data_Usg_M09,seconds_of_data_norm,seconds_of_data_log
0,471.0,0.0,0.0,1693.8,44.980000,55.000000,4.0,7.0,b'prime',b'Indirect',...,5.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,4.110874
1,1057.0,0.0,0.0,38682.5,44.989330,47.000000,3.0,5.0,b'prime',b'Indirect',...,5.0,115.0,0.0,0.0,0.0,0.0,330.0,646.0,48300.0,10.785208
2,1177.0,0.0,0.0,5526.4,51.775434,46.176471,7.0,4.8,b'prime',b'Retail',...,5.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,14700.0,9.595671
3,1181.0,0.0,0.0,505.6,51.775434,75.000000,7.0,4.8,b'prime',b'Retail',...,5.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,1185.0,0.0,1.0,9143.2,83.320000,50.000000,5.0,3.0,b'prime',b'Indirect',...,5.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,15840.0,9.670357
5,1207.0,0.0,1.0,2930.7,62.806660,56.000000,7.0,3.0,b'prime',b'Indirect',...,5.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,24240.0,10.095801
6,1210.0,0.0,0.0,1863.5,79.615000,64.000000,11.0,2.0,b'other',b'Retail',...,5.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,11340.0,9.336180
7,1211.0,0.0,0.0,9368.3,66.643330,45.000000,9.0,9.0,b'prime',b'Indirect',...,5.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,23880.0,10.080838
8,1213.0,0.0,0.0,10357.0,51.775434,46.176471,7.0,4.8,b'prime',b'Retail',...,5.0,55.0,0.0,0.0,0.0,604.0,148.0,0.0,24000.0,10.085851
9,1214.0,0.0,0.0,3311.2,71.083330,54.000000,4.0,3.0,b'prime',b'National Sales',...,5.0,143.0,0.0,0.0,0.0,0.0,0.0,0.0,22051.0,10.001159


In [21]:
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H4(children='Telco Customers data history'),
    generate_table(comms_df.iloc[:, :10])
])

if __name__ == '__main__':
    #from waitress import serve
    #serve(app , host="127.0.0.1", port=2525)
    app.run_server(host="0.0.0.0", port=8050, debug=True)

Dash app running on http://0.0.0.0:8050/
